# Claycode SIGGRAPH Evaluation

In [1]:
import pandas as pd
import plotly.express as px

df = pd.read_csv('results.csv')
df['type'] = df['filename'].str.split('-').map(lambda x: "-".join(x[:-1]).replace("_clay","clay"))
df = df[['successful', 'wave_amplitude', 'type']]

grouped = df.groupby(['wave_amplitude', 'type'])
df = grouped['successful'].agg(['sum', 'count']).reset_index()
df['success_rate'] = df['sum'] / df['count']

fig = px.line(
    df,
    x="wave_amplitude",
    y="success_rate",
    color="type",
    line_dash="type",  # Add different dash styles based on 'type'
    markers=True,
    title="Success Rate vs Wave Amplitude",
    labels={"wave_amplitude": "Wave Amplitude", "success_rate": "Success Rate", "type": "Type"}
)

fig.update_traces(opacity=0.7)  # Make lines semi-transparent

fig.show()

In [22]:
# y: every kind of string (group by message name)
# x: amplitudine
# y: type of code

df = pd.read_csv('results.csv')

# Extract the string between the last '-' and '.png' in the filename
df['extracted_value'] = df['filename'].str.extract(r'-([^-]+)\.png$')

# Calculate the length of the extracted value and use it for the Y-axis
df['extracted_value_len'] = df['extracted_value'].apply(len)

# Sort by the length of the extracted value
df = df.sort_values('extracted_value_len')

# Group by 'extracted_value_len' and 'wave_amplitude', then count the successful entries
grouped_df = df.groupby(['extracted_value_len', 'wave_amplitude'], as_index=False).agg({
    'successful': 'sum'  # Count of successful rows (True = 1, False = 0)
})

x_max = grouped_df['wave_amplitude'].max()
if x_max != 1:
    x_max = 1

# Create the heatmap with length of extracted value on Y-axis
fig = px.density_heatmap(
    grouped_df,
    x='wave_amplitude',
    y='extracted_value_len',
    z='successful',
    title='Heatmap of Wave Amplitude and Length of Extracted Filename Value (Count of Successful)',
    labels={
        'wave_amplitude': 'Wave Amplitude',
        'extracted_value_len': 'Length of Extracted Value',
        'successful': 'Count of Successful Entries'
    },
    color_continuous_scale='Teal',
)

fig.update_layout(
    xaxis=dict(range=[0, 1]),
)

# Show the plot
fig.show()